Authentication

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%load_ext google.colab.data_table

In [12]:
project_id = 'exerciseday1'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/Nested Queries**

kode berikut digunakan untuk mengambil info stasiun awal dan stasiun akhir pada tanggal 8 Februari 2018 dengan start stasiun yang memiliki kios

In [17]:
df = client.query('''
SELECT
  start_station_name,
  end_station_name
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
WHERE
  TIMESTAMP_TRUNC(start_date, DAY) = TIMESTAMP("2018-02-08")
  AND start_station_id IN (
    SELECT
      CAST(station_id AS INT64)
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
    WHERE
      has_kiosk = TRUE
  )
LIMIT 3;''').to_dataframe()

df.head()

,start_station_name,end_station_name
0,10th St at Fallon St,2nd Ave at E 18th St
1,11th St at Natoma St,Clay St at Battery St
2,12th St at 4th Ave,Lake Merritt BART Station


**Joins**

menggunakan inner join untuk  melihat lebih detail, info dari kapasitas stasiun start dengan data yang ada pada tanggal 8 Februari 2018

In [18]:
df = client.query('''
SELECT
  trips.start_station_name,
  trips.end_station_name,
  station_info.name AS station_full_name,
  station_info.capacity AS station_capacity
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` AS trips
INNER JOIN
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS station_info
ON
  trips.start_station_id = CAST(station_info.station_id AS INT64)
WHERE
  TIMESTAMP_TRUNC(trips.start_date, DAY) = TIMESTAMP("2018-02-08")
LIMIT 3;
''').to_dataframe()

df.head()

,start_station_name,end_station_name,station_full_name,station_capacity
0,El Embarcadero at Grand Ave,10th St at Fallon St,El Embarcadero at Grand Ave,23
1,Central Ave at Fell St,11th St at Bryant St,Central Ave at Fell St,31
2,Dolores St at 15th St,11th St at Bryant St,Dolores St at 15th St,19


**UNIONS**

union start station id pada tabel bike_trips dengan station id pada tabel bike_station_info

In [19]:
df = client.query('''
(SELECT
  start_station_id
FROM
 `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT
  3)
UNION DISTINCT
(SELECT
  CAST(station_id AS INT64)
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
LIMIT
  3)''').to_dataframe()

df.head()

,start_station_id
0,222
1,523
2,501
3,504


**Aggregate Functions**

In [21]:
df = client.query('''
SELECT
  start_station_name,
  COUNT(start_station_id) AS total_start_trip
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
WHERE
  TIMESTAMP_TRUNC(start_date, DAY) = TIMESTAMP("2018-02-08")
  AND start_station_id IN (
    SELECT
      CAST(station_id AS INT64)
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
    WHERE
      has_kiosk = TRUE
  )
GROUP BY
  start_station_name
LIMIT 3;''').to_dataframe()

df.head()

,start_station_name,total_start_trip
0,10th St at Fallon St,10
1,2nd Ave at E 18th St,24
2,24th St at Chattanooga St,18


**Window Function**

In [23]:
df = client.query('''
SELECT
  start_station_name,
  total_start_trip,
  ROW_NUMBER() OVER (ORDER BY total_start_trip DESC) AS rank
FROM (
  SELECT
    start_station_name,
    COUNT(start_station_id) AS total_start_trip
  FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
  WHERE
    TIMESTAMP_TRUNC(start_date, DAY) = TIMESTAMP("2018-02-08")
    AND start_station_id IN (
      SELECT
        CAST(station_id AS INT64)
      FROM
        `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
      WHERE
        has_kiosk = TRUE
    )
  GROUP BY
    start_station_name
)
LIMIT 3;
''').to_dataframe()

df.head()


,start_station_name,total_start_trip,rank
0,San Francisco Caltrain (Townsend St at 4th St),125,1
1,Berry St at 4th St,116,2
2,Market St at 10th St,110,3


**Common Table Expressions**

In [25]:
df = client.query('''
WITH
  bikeshare_trip AS (
  SELECT
    start_station_name,
    end_station_name
  FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
  WHERE
    TIMESTAMP_TRUNC(start_date, DAY) = TIMESTAMP("2018-02-08"))
  SELECT
    bikeshare_trip.start_station_name,
    bikeshare_trip.end_station_name
  FROM
    bikeshare_trip
  LIMIT 3;
  ''').to_dataframe()

df.head()

,start_station_name,end_station_name
0,10th St at Fallon St,10th Ave at E 15th St
1,10th St at Fallon St,10th St at Fallon St
2,2nd Ave at E 18th St,10th St at Fallon St


**PIVOT & DATE AND TIME**

In [29]:
%%bigquery --project exerciseday1
SELECT
  TIMESTAMP_TRUNC(start_date, DAY) AS trip_date,
  start_station_name,
  COUNT(*) AS total_trips
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  trip_date,
  start_station_name
ORDER BY
  trip_date,
  start_station_name
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,trip_date,start_station_name,total_trips
0,2013-08-29 00:00:00+00:00,2nd at Folsom,12
1,2013-08-29 00:00:00+00:00,2nd at South Park,11
2,2013-08-29 00:00:00+00:00,2nd at Townsend,8


**STRING MANIPULATION**

In [30]:
%%bigquery --project exerciseday1
SELECT
  DISTINCT CONCAT(start_station_name, end_station_name) AS trips
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,trips
0,10th St at Fallon St10th Ave at E 15th St
1,12th St at 6th Ave10th Ave at E 15th St
2,13th St at Franklin St10th Ave at E 15th St


**CASE STATEMENT**

In [34]:
%%bigquery --project exerciseday1
SELECT
  start_station_name,
  member_birth_year,
  CASE
    WHEN member_birth_year >= 1965 THEN 'Young and Children Biker'
    ELSE 'Mature Biker'
END
  AS member_age_category
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,start_station_name,member_birth_year,member_age_category
0,10th Ave at E 15th St,1984,Mature Biker
1,10th Ave at E 15th St,<NA>,Adult or Children Biker
2,10th St at Fallon St,1984,Mature Biker


**UDF**

In [35]:
%%bigquery --project exerciseday1
CREATE TEMP FUNCTION
  getAge(arr ANY TYPE) AS (
    CASE
    WHEN arr >= 1965 THEN 'Young and Children Biker'
    ELSE 'Mature Biker'
  END
  );
SELECT
  member_birth_year,
  getAge(member_birth_year) AS age
FROM (
  SELECT
  member_birth_year
  FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
  LIMIT 3)

Query is running:   0%|          |

Downloading:   0%|          |

,member_birth_year,age
0,1984,Mature Biker
1,<NA>,Adult or Children Biker
2,1984,Mature Biker
